In [1]:
import pandas as pd
import numpy as np
import math
import gc

import warnings
warnings.filterwarnings("ignore")

In [2]:
### H1 Resort hotel
### H2 City hotel
dataset = pd.read_csv("D:\Book\H2.csv")
dataset.shape

(79330, 31)

In [3]:
#fill na with 0 for children
dataset['Children'].fillna(0,inplace=True)

# create arrival date as ArrivalDate
monthmap = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
dataset['ArrivalDateMonthNumber'] = dataset['ArrivalDateMonth'].map(monthmap)
dataset['ArrivalDate'] = pd.to_datetime(dataset['ArrivalDateDayOfMonth'].astype(str) + '-' + dataset['ArrivalDateMonth'].astype(str) + '-'+ dataset['ArrivalDateYear'].astype(str))

#create booking date
dataset['BookingDate'] = dataset['ArrivalDate'] - pd.to_timedelta(dataset['LeadTime'], unit='d')

#create stay days
dataset['StaysInDays'] = dataset['StaysInWeekNights'] + dataset['StaysInWeekendNights']

#Change format and make it pandas date
dataset['ReservationStatusDate']=pd.to_datetime(dataset['ReservationStatusDate'])
#dataset['StaysInDaysCalc'] = (dataset['ReservationStatusDate'] - dataset['ArrivalDate']).dt.days

#Remove no-show, as we don't know for how many nights it was reserved. Size of the data is also very small.
#Remove cases where there are no guests. both adults and children is 0.
#We are only concerned with Transient bookings, as this is the major source of revenue for hotels
#For ADR, we will restrict with values above 5 percentile and below 99.99 percentile. rest of the data looks like outlier. As we do not have means to get clarification for data, we will remove data beyond this. 

dataset=dataset[(dataset['ReservationStatus']!='No-Show') & ((dataset['Adults']>0) | (dataset['Children']>0)) & (dataset['CustomerType']=='Transient') & (dataset['ADR']<=np.percentile(dataset['ADR'],99.99)) & (dataset['ADR']>np.percentile(dataset['ADR'],5))]
dataset.reset_index(inplace=True)

#If a transaction reflects multiple nights stay, we will add the nights for the following arrival days and add number of days for lead time as well.
#So 2 new columns AdjustedArrivalDate and AdjustedLeadTime will be created.

dataset = dataset.loc[dataset.index.repeat(dataset['StaysInDays'])].assign(CarryForwardForFutureDays=1)
dataset['KeepIndex'] = dataset.index.values
dataset['AdjustBy'] = dataset.groupby(['KeepIndex'])['CarryForwardForFutureDays'].cumsum()-1
dataset['AdjustedArrivalDate'] = dataset['ArrivalDate'] + pd.to_timedelta(dataset['AdjustBy'], unit='d')
dataset['AdjustedLeadTime'] = dataset['LeadTime'] + dataset['AdjustBy']
dataset.reset_index(inplace=True)
dataset=dataset[['IsCanceled', 'AdjustedLeadTime', 'LeadTime', 'ArrivalDateYear','ArrivalDateMonth', 'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth','StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children','Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel','IsRepeatedGuest', 'PreviousCancellations','PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType','BookingChanges', 'DepositType', 'Agent', 'Company','DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces','TotalOfSpecialRequests', 'ReservationStatus', 'ReservationStatusDate','ArrivalDateMonthNumber', 'ArrivalDate', 'BookingDate', 'StaysInDays','AdjustedArrivalDate']]

## We will calculate how many rooms were booked using below method.

# keep 0 as placeholder in new column
dataset['NumberOfRooms'] = 0

## One room can accomodate 2 adults and 2 kid in a hotel.
#If a transaction has 2 adult and 2 kid, it will be considered as 1 room. If a transaction has 1 adult and 2 kid, 2 adult and 1 kid, or 1 adult and 1 kid, it will be considered as 1 room.
# Room =1
# If adults less than or equal to 2 and 2 children. Or no adult and less than or equal to 2 children.
dataset['NumberOfRooms'][((dataset['Adults']<=2) & (dataset['Children']<=2))] = 1

## For any other transaction, we will follow below rules.
#divide by 2 and round up, if no kids or no adults
dataset['NumberOfRooms'][(dataset['Adults']>2) & (dataset['Children']==0)] = np.round(dataset['Adults']/2)

#3 adult with <=2 kid, 2 room
dataset['NumberOfRooms'][((dataset['Adults']==3) & (dataset['Children']<=2))] = 2

#<=2 adult with 3 kid, 2 room
dataset['NumberOfRooms'][((dataset['Adults']<=2) & (dataset['Children']==3))] = 2

############################

In [4]:

###############create dataframe in desired format for modelling
def create_desired_df(dataset2_aggregation_df):
    sdate = dataset2_aggregation_df['AdjustedArrivalDate'].min()
    edate = dataset2_aggregation_df['AdjustedArrivalDate'].max()
    date_range = list(pd.date_range(sdate,edate,freq='d'))

    desired_df = pd.DataFrame({'AdjustedLeadTime':list(i for i in range(dataset2_aggregation_df['AdjustedLeadTime'].max()+1,-1,-1))})
    desired_df = desired_df.loc[np.tile(desired_df.index,len(date_range))].assign(AdjustedArrivalDate=np.repeat(date_range,len(desired_df)))

    desired_df.reset_index(inplace=True,drop=True)
    return desired_df

def create_decode_dict(desired_df,dataset2_aggregation_df):
    desired_df=desired_df.merge(dataset2_aggregation_df,how='left')
    desired_df.fillna(0,inplace=True)

    reverse_list=list(i for i in range(dataset2_aggregation_df['AdjustedLeadTime'].max()+1,-1,-1))
    correct_list=list(i for i in range(0,dataset2_aggregation_df['AdjustedLeadTime'].max()+2,1))

    recode_dict={}
    for i,j in zip(reverse_list,correct_list):
        recode_dict[i]=j
    return recode_dict

def create_cumulative(desired_df,dataset2_aggregation_df,NumberOfRooms,recode_dict):
    ## merge both in single dataframe to get booking and cancellation data at arrival date level for lead time
    desired_df=desired_df.merge(dataset2_aggregation_df,how='left')
    desired_df.fillna(0,inplace=True)



    desired_df['AdjustedLeadTime'] = desired_df['AdjustedLeadTime'].map(recode_dict)

    desired_df2=desired_df.groupby(['AdjustedArrivalDate', 'AdjustedLeadTime']).sum() \
      .groupby(level=0).cumsum().reset_index()
    desired_df['Cumulative'+NumberOfRooms] = desired_df2[NumberOfRooms]
    desired_df['Cumulative'+NumberOfRooms].fillna(0,inplace=True)
    
    desired_df['AdjustedLeadTime'] = desired_df['AdjustedLeadTime'].map(recode_dict)

    return desired_df


def create_data(dataset,keep_columns,calc_column):
    dataset2=dataset[keep_columns]

    ## get reservation summed by arrival date, at lead time level
    dataset2_aggregation=dataset2[['AdjustedArrivalDate','AdjustedLeadTime',calc_column]].groupby(['AdjustedArrivalDate','AdjustedLeadTime']).sum()
    dataset2_aggregation_df = pd.DataFrame({'AdjustedArrivalDate':dataset2_aggregation.index.get_level_values(0).to_list(),
    'AdjustedLeadTime':dataset2_aggregation.index.get_level_values(1).to_list(),
    calc_column:dataset2_aggregation.values.ravel()})

    ## get cancellation aggregation by arrival date, at ReservationStatusDate level by aggregating cancellation at the lead time level when they are canelled
    dataset2_cancel = dataset2[dataset2['IsCanceled']==1]
    dataset2_cancel.reset_index(inplace=True,drop=True)

    dataset2_cancel['AdjustedLeadTime'] = (dataset2_cancel['AdjustedArrivalDate']-dataset2_cancel['ReservationStatusDate']).dt.days
    dataset2_cancel_aggregation = dataset2_cancel[['AdjustedArrivalDate','AdjustedLeadTime',calc_column]].groupby(['AdjustedArrivalDate','AdjustedLeadTime']).sum()
    dataset2_cancel_aggregation_df = pd.DataFrame({'AdjustedArrivalDate':dataset2_cancel_aggregation.index.get_level_values(0).to_list(),
    'AdjustedLeadTime':dataset2_cancel_aggregation.index.get_level_values(1).to_list(),
    calc_column+'_Substract':dataset2_cancel_aggregation.values.ravel()})

    desired_df1 = create_desired_df(dataset2_aggregation_df)
    recode_dict = create_decode_dict(desired_df1,dataset2_aggregation_df)
    desired_df2 = create_desired_df(dataset2_aggregation_df)

    desired_df1 = create_cumulative(desired_df1,dataset2_aggregation_df,calc_column,recode_dict)
    desired_df2 = create_cumulative(desired_df2,dataset2_cancel_aggregation_df,calc_column+'_Substract',recode_dict)

    desired_df=desired_df1.merge(desired_df2,how='left')
    #     desired_df.fillna(0,inplace=True)

    desired_df['Cumulative'+calc_column+'Net'] = desired_df['Cumulative'+calc_column] - desired_df['Cumulative'+calc_column+'_Substract']

    final_output = desired_df[desired_df['AdjustedLeadTime']==0][['AdjustedArrivalDate','AdjustedLeadTime','Cumulative'+calc_column+'Net']]
    final_output.rename(columns={'Cumulative'+calc_column+'Net':'TotalRooms'},inplace=True)

    desired_df=desired_df.merge(final_output,how='left')
    desired_df['TotalRooms'].fillna(method='bfill',inplace=True)
    return desired_df

#########################





In [5]:
###number of rooms
keep_columns=['AdjustedArrivalDate','NumberOfRooms','BookingDate','AdjustedLeadTime','ReservationStatusDate','IsCanceled']
desired_df = create_data(dataset,keep_columns,calc_column='NumberOfRooms')
#keep data for less than or equal to 100 Adjusted lead time
desired_df = desired_df[(desired_df['AdjustedLeadTime'] <= 100)]
desired_df.reset_index(inplace=True,drop=True)


dataset['Revenue'] = dataset['ADR'] * dataset['NumberOfRooms']
keep_columns=['AdjustedArrivalDate','Revenue','BookingDate','AdjustedLeadTime','ReservationStatusDate','IsCanceled']
desired_df2 = create_data(dataset,keep_columns,calc_column='Revenue')
desired_df2.rename(columns={'TotalRooms':'TotalRevenue'},inplace=True)
desired_df2 = desired_df2[(desired_df2['AdjustedLeadTime'] <= 100)]
desired_df2.reset_index(inplace=True,drop=True)
del desired_df2['TotalRevenue']

desired_df = desired_df.merge(desired_df2,how='left')

##create seasonality columns

#day of the week
desired_df['DayOfWeek'] = desired_df['AdjustedArrivalDate'].dt.day_name()

#weekend
desired_df['Weekend'] = np.where(desired_df['DayOfWeek'].isin(['Sunday', 'Saturday']), 1,0)

#
daymap = {'Monday':1,'Tuesday':2,'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
desired_df['DayOfWeek'] = desired_df['DayOfWeek'].map(daymap)

#week of the year
desired_df['WeekOfYear'] = desired_df['AdjustedArrivalDate'].dt.week

#day of the month
desired_df['DayOfMonth'] = desired_df['AdjustedArrivalDate'].dt.day

#week of the month
desired_df['WeekOfMonth'] = pd.to_numeric(desired_df['DayOfMonth']/7)
desired_df['WeekOfMonth'] = desired_df['WeekOfMonth'].apply(lambda x: math.ceil(x))

#month of the year
desired_df['MonthOfYear'] = desired_df['AdjustedArrivalDate'].dt.month

In [6]:
desired_df.to_csv("D:\Book\H2_PredictingOccupancy_100daysPrior.csv",index=False)

In [7]:
desired_df.head()

,AdjustedLeadTime,AdjustedArrivalDate,NumberOfRooms,CumulativeNumberOfRooms,NumberOfRooms_Substract,CumulativeNumberOfRooms_Substract,CumulativeNumberOfRoomsNet,TotalRooms,Revenue,CumulativeRevenue,Revenue_Substract,CumulativeRevenue_Substract,CumulativeRevenueNet,DayOfWeek,Weekend,WeekOfYear,DayOfMonth,WeekOfMonth,MonthOfYear
0,100,2015-07-01,0.0,74.0,0.0,0.0,74.0,65.0,0.0,7180.0,0.0,0.0,7180.0,3,0,27,1,1,7
1,99,2015-07-01,0.0,74.0,0.0,0.0,74.0,65.0,0.0,7180.0,0.0,0.0,7180.0,3,0,27,1,1,7
2,98,2015-07-01,0.0,74.0,0.0,0.0,74.0,65.0,0.0,7180.0,0.0,0.0,7180.0,3,0,27,1,1,7
3,97,2015-07-01,0.0,74.0,0.0,0.0,74.0,65.0,0.0,7180.0,0.0,0.0,7180.0,3,0,27,1,1,7
4,96,2015-07-01,0.0,74.0,0.0,0.0,74.0,65.0,0.0,7180.0,0.0,0.0,7180.0,3,0,27,1,1,7
